In [21]:
import pickle
import pandas as pd

from numpy import array
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add

In [22]:
images_folder = 'D:/datasets/flickr-30k-images'

curr_folder = "D:/YandexDisk/datasets/"
path_train = curr_folder + "captions-ru-train.csv"
path_test = curr_folder + "captions-ru-test.csv"

path_features = curr_folder + "features.pkl"
path_sentences = curr_folder + "sentences-train.pkl"

path_model = curr_folder + "model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5"

# Загрузка данных

Мы собираемся обучить данные по всем фотографиям и подписям в обучающем наборе данных. Во время обучения мы будем отслеживать производительность модели в наборе данных разработки и использовать эту производительность, чтобы решить, когда сохранять модели в файл.

Модель, которую мы разработаем, будет генерировать подпись к фотографии, и подпись будет генерироваться по одному слову за раз.

Последовательность ранее сгенерированных слов будет предоставлена в качестве входных данных. Поэтому нам понадобится "первое слово", чтобы начать процесс генерации, и "последнее слово", чтобы сигнализировать об окончании подписи. Для этой цели мы будем использовать строки "startseq" и "endseq". Эти маркеры добавляются к загруженным описаниям по мере их загрузки. Важно сделать это сейчас, прежде чем мы закодируем текст, чтобы токены также были закодированы правильно.

# Закодировать знаки в числа

Текст описания необходимо будет закодировать в числа, прежде чем его можно будет представить модели в качестве входных данных или сравнить с предсказаниями модели.

Первым шагом в кодировании данных является создание согласованного сопоставления слов с уникальными целочисленными значениями. Keras предоставляет класс Tokenizer, который может изучить это сопоставление из загруженных данных описания.

Каждое описание будет разделено на слова. Модель будет предоставлена одним словом и фотографией и сгенерирует следующее слово. Затем первые два слова описания будут предоставлены модели в качестве входных данных вместе с изображением для создания следующего слова. Именно так будет обучаться модель.

# Создание последовательности

Приведенная ниже функция с именем create_sequences(), учитывая токенизатор, максимальную длину последовательности и словарь всех описаний и фотографий, преобразует данные в пары ввода-вывода данных для обучения модели.

В модели есть два входных массива: один для признаков фотографии и один для закодированного текста. Существует один вывод для модели, который представляет собой закодированное следующее слово в текстовой последовательности.

Входной текст кодируется в виде целых чисел, которые будут передаваться на слой встраивания слов. Признаки изображения будут передаваться непосредственно в другую часть модели. Модель выдаст прогноз, который будет представлять собой распределение вероятностей по всем словам в словаре.

Таким образом, выходные данные будут представлять собой однократно закодированную версию каждого слова, представляющую идеализированное распределение вероятностей со значениями 0 во всех позициях слов, кроме фактической позиции слова, которая имеет значение 1.

In [23]:
def image_names_set(data):
    vals = set()

    for idx in data.index:
        vals.add(data.iat[idx, 0][:-4])

    return vals

def load_image_features(filename, data):
    all_features = pickle.load(open(filename, 'rb'))
    features = {k: all_features[k] for k in data}

    return features

def create_tokenizer(data):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data)

    return tokenizer

def find_max_length(data):
    max_len = 0

    for idx in data.index:
        curr_len = len(data.iat[idx, 2])
        if curr_len > max_len:
            max_len = curr_len

    return max_len

def create_sequences(tokenizer, max_length, data, image_name, vocab_size):
    X1, X2, y = list(), list(), list()

    for idx, row in data.iterrows():
        seq = tokenizer.texts_to_sequences([row['comment']])[0]
        
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]

            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

            X1.append(image_name)
            X2.append(in_seq)
            y.append(out_seq)

    return array(X1), array(X2), array(y)

# Построение модели

Мы опишем модель в трёх частях:

1 – Извлечение признаков изображения. Это 16-слойная модель VGG, предварительно обученная на наборе данных ImageNet. Мы предварительно обработали изображения с помощью модели VGG (без выходного слоя) и будем использовать извлечённые признаки, предсказанные этой моделью, в качестве входных данных.

2 – Обработка последовательностей. Это слой встраивания слов для обработки ввода текста, за которым следует слой рекуррентной нейронной сети с длительной кратковременной памятью (LSTM).

3 – Расшифровка. (1) и (2) выводят вектор фиксированной длины. Они объединяются вместе и обрабатываются плотным слоем, чтобы сделать окончательный прогноз.

Модель (1) ожидает, что входные признаки изображений будут вектором из 4096 элементов. Они обрабатываются плотным слоем для получения 256-элементного представления изображения.

Модель (2) ожидает входные последовательности с заранее определённой длиной, которые подаются в слой встраивания, использующий маску для игнорирования дополненных значений. За этим следует слой LSTM с 256 единицами памяти.

Обе входные модели создают вектор из 256 элементов. Кроме того, обе входные модели используют регуляризацию в виде 50% отсева (dropout). Это делается для того, чтобы уменьшить переобучение модели на текущем наборе данных, так как эта конфигурация модели обучается очень быстро.

Модель (3) объединяет векторы из обеих входных моделей с помощью операции сложения. Затем этот вектор подаётся на плотный слой из 256 нейронов, а затем на конечный выходной плотный слой, который делает прогноз softmax по всему выходному словарю для следующего слова в последовательности.

# Генератор данных

Генератор данных будет выдавать данные на одно изображении в каждой партии. Это будут все последовательности, сгенерированные для изображения и её набора описаний.

Функция data_generator() будет генератором данных и будет принимать загруженные текстовые описания, признаки изображений, токенизатор и максимальную длину.8 ГБ оперативной памяти должно быть более чем достаточно.

Вы можете видеть, что мы вызываем функцию create_sequence (), чтобы создать пакет данных для одного изображения, а не для всего набора данных. Это означает, что мы должны обновить функцию create_sequences (), чтобы удалить “итерацию по всем описаниям” для цикла.

Генератор данных, предназначен для использования в вызове model.fit_generator().

Обратите внимание, что это очень простой генератор данных. Большая экономия памяти, которую он предлагает, заключается в том, чтобы не иметь развернутых последовательностей обучающих и тестовых данных в памяти до подгонки модели, чтобы эти образцы (например, результаты create_sequences()) создавались по мере необходимости для каждого изображения.

Некоторые нестандартные идеи для дальнейшего совершенствования этого генератора данных включают в себя:
– Рандомизируйте порядок фотографий каждой эпохи.
– Работайте со списком идентификаторов изображений и загружайте текст и данные изображений по мере необходимости, чтобы ещё больше сократить объём памяти.
– Получите более одного изображения в партии.

In [24]:
def build_model(vocab_size, max_length):
    inputs1 = Input(shape=(4096,))

    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    de1 = add([fe2, se3])
    de2 = Dense(256, activation='relu')(de1)
    outputs = Dense(vocab_size, activation='softmax')(de2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

def data_generator(tokenizer, max_length, data, images, vocab_size):
    while 1:
        start_index = 0
        end_index = len(data) - 1 - 5
        
        for idx in range(start_index, end_index+1, 5):
            five_rows = data[idx:idx+5]
            image_name = data.iat[idx, 0][:-4]
            image = images[image_name][0]

            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, five_rows, image, vocab_size)

            yield [in_img, in_seq], out_word

# Обучение

В этом примере мы отбросим загрузку тестового набора данных и контрольные точки модели и просто сохраним модель после каждой эпохи обучения. Затем мы сможем вернуться и загрузить/оценить каждую сохраненную модель после обучения, чтобы найти ту, которая имеет наименьшие потери.

In [25]:
train_df = pd.read_csv(path_train, delimiter='|')
train_features = load_image_features(path_features, image_names_set(train_df))
print('кол-во подписей .............. %d' % len(train_df))

with open (path_sentences, 'rb') as f:
    sentences = pickle.load(f)
tokenizer = create_tokenizer(sentences)
vocab_size = len(tokenizer.word_index) + 1
print('размер словаря предложений ... %d' % vocab_size)

max_length = find_max_length(train_df)
print('длина подписи ................ %d' % max_length)

кол-во подписей .............. 54510
размер словаря предложений ... 26817
длина подписи ................ 400
